# Sarukhanian Construction Demo
This notebook reproduces the Maple-inspired Sarukhanian block construction, computes nonperiodic autocorrelations (NPAFs), and experiments with tweaks that repair the zero-sum property.

In [ ]:
from pathlib import Path
import numpy as np

from src.sequences import BASE_SEQUENCES, PATTERN_COLUMNS
from src.construction import (
    build_sarukhanian_110,
    get_default_plan,
    plan_to_sequences,
    verify_four_sequences,
)
from src.npaf import npaf_all_shifts, npaf_sum_four
from src.repair import apply_sign_flip, swap_blocks, auto_local_search
from src.viz import plot_npaf_series, plot_sum_series, REPORT_FIG_DIR

REPORT_FIG_DIR.mkdir(parents=True, exist_ok=True)
report_dir = REPORT_FIG_DIR.resolve()
report_dir

## Base sequences and pattern columns
These are the canonical BS(3,2) rows and Sarukhanian column patterns used in the block plan.

In [ ]:
base_info = {name: seq.tolist() for name, seq in BASE_SEQUENCES.items()}
pattern_info = {name: vec.tolist() for name, vec in PATTERN_COLUMNS.items()}
base_info, pattern_info

## Baseline construction (buggy reference)
The default plan mirrors the Maple ordering and yields four length-110 sequences whose summed NPAFs are not yet zero.

In [ ]:
plan = get_default_plan()
x, y, z, w = build_sarukhanian_110({"plan": plan})
tuple(seq.size for seq in (x, y, z, w))

In [ ]:
npaf_curves = {label: npaf_all_shifts(seq) for label, seq in zip(('X', 'Y', 'Z', 'W'), (x, y, z, w))}
sum_series = npaf_sum_four(x, y, z, w)
diag = verify_four_sequences(x, y, z, w)
print(diag)
paths = []
for label, seq in zip(('X', 'Y', 'Z', 'W'), (x, y, z, w)):
    paths.append((label, plot_npaf_series(seq, f"Baseline {label}")))
sum_path = plot_sum_series(sum_series, "Baseline summed NPAF")
paths, sum_path

## Manual tweaks (sign flips and swaps)
Use the repair helpers to explore nearby plans and see whether diagnostics improve.

In [ ]:
tweaked = apply_sign_flip(plan, where=0)
tweaked = swap_blocks(tweaked, 1, 2)
t_x, t_y, t_z, t_w = plan_to_sequences(tweaked).as_tuple()
t_diag = verify_four_sequences(t_x, t_y, t_z, t_w)
t_sum = npaf_sum_four(t_x, t_y, t_z, t_w)
manual_path = plot_sum_series(t_sum, "Manual tweak summed NPAF")
t_diag, manual_path

## Local search experiment
Run a bounded stochastic search over sign flips/swaps.

In [ ]:
search_result = auto_local_search(plan, max_steps=50, random_seed=42)
best_diag = search_result['diagnostics']
best_sum = verify_four_sequences(*search_result['sequences'])['sum_series']
best_path = plot_sum_series(best_sum, "Search best summed NPAF")
best_diag, best_path

## What I learned
- The Maple-derived block order leaves a handful of stubborn non-zero shifts.
- Sign flips on early blocks change only a few shifts; deeper plan edits or search are needed for full cancellation.
- Automated local search provides a repeatable knob to hunt for zero-sum repairs.